In [1]:
import requests

In [2]:
from flask import Flask, request, jsonify
from transformers import GPTNeoForCausalLM, AutoTokenizer

# Load the GPT-4 model and tokenizer
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")

app = Flask(__name__ if __name__ == "__main__" else "dummy")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
def get_weather_from_opendata(api_key, location=None):

  api_key = '8f54f89d3afb9bf113bef482107e8c7f'
  base_url = "https://api.openweathermap.org/data/2.5/weather?"
  url = f"{base_url}appid={api_key}&q={location}&units=metric"

  try:
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()
    return {
      "temperature": data["main"]["temp"],
      "humidity": data["main"]["humidity"],
    }
  except requests.exceptions.RequestException as e:
    print(f"Error retrieving weather data: {e}")
    return None

In [4]:
def air_conditioner_control(desired_temperature, threshold=20):
  weather = get_weather_from_opendata('8f54f89d3afb9bf113bef482107e8c7f', "Taipei")  # Replace with your API key and city

  if weather is None:
    print("Error getting weather data. Using default temperature.")
    temperature = 18  # Default temperature if weather retrieval fails
  else:
    temperature = weather["temperature"]

  if desired_temperature < threshold:
    print(f"Current temperature: {temperature:.1f}°C")
   # print("Turn on the air conditioner")
    return "Turn on the air conditioner"
  else:
    print(f"Current temperature: {temperature:.1f}°C")
   # print("Turn off the air conditioner")
    return "Turn off the air conditioner"

# Get user input for desired temperature
desired_temp = 18

# Call the air conditioner control function with desired temperature
action = air_conditioner_control(desired_temp)

print(action)

Current temperature: 25.7°C
Turn on the air conditioner


In [10]:
@app.route("/generate_text", methods=["POST"])

def generated_text():
  prompt = request.json.get("prompt")
  if prompt is None:
    return jsonify({"error": "Missing prompt in request body"}), 400

  input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]

  output = model.generate(
      input_ids=input_ids,
      max_length=50,
      num_return_sequences=1,
      temperature=0.7
  )

  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

  return jsonify({"generated_text": generated_text})

@app.route("/air_conditioner", methods=["GET"])
def get_ac_recommendation():
  desired_temp = 18  # Adjust desired temperature

  action = air_conditioner_control(desired_temp)

  return jsonify({"action": action})

In [11]:
if __name__ == "__main__":
  app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
